# Поиск критериев платежеспособности клиентов банка

Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов и обезличенные данные по таким параметрам, как количество детей, рабочий стаж, возраст, образование, семейное положение, пол, должность, доход и цель кредитования, а также отметка о возврате кредита в срок. 

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

1. [Получение данных](#step1)  
2. [Предобработка данных](#step2)
3. [Категоризация кредитных целей и клиентов](#step3)
4. [Критерии оценки платежеспособности клиентов банка](#step4)
5. [Результаты исследования](#step5)

##  Получение данных <a id="step1"></a>   

Изучим данные, предоставленные кредитным отделом банка.

### Импорт библиотек

In [1]:
! pip install pandas -U
! pip install pymystem3 -U

import pandas as pd
from pymystem3 import Mystem
m = Mystem() 

import warnings
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pandas in /home/jovyan/.local/lib/python3.7/site-packages (1.3.4)
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pymystem3 in /opt/conda/lib/python3.7/site-packages (0.2.0)


Прочитаем файл *data.csv* и сохраним его в переменной *df*. 

In [2]:
df = pd.read_csv('/datasets/data.csv')

Получим первые строки таблицы.

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Общая информация о данных таблицы *df*.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, тип данных - **float64**, **int64**, **object**.

Подробно разберём, какие в *df* столбцы и какую информацию они содержат:

* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пустые значения.

### Выводы

Каждая строка таблицы содержит информацию об отдельном заемщике: количество детей в семье, общий трудовой стаж, возраст, образование, семейное положение, пол, тип занятости, задолжность по возврату кредитов, ежемесячный доход, цель получения кредита. Проблемы, которые нужно решать: пропуски, мусор в столбце *days_employed*, разный регистр. Для проверки рабочих гипотез особенно ценны столбцы *children*, *family_status*, *debt*, *total_income*, *purpose*.

## Предобработка данных <a id="step2"></a>  

Исключим пропуски, заменим вещественный тип данных на целочисленный, проверим данные на наличие дубликатов, артефактов.

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
df.isnull().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверим, соответствуют ли пропуски в столбце *days_employed* пропускам в столбце *total_income*.

In [6]:
print(df[(df['days_employed'] == 'NaN') & (df['total_income'] == 'NaN')])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Датафрейм пуст: фильтр вернул те строки, в которых в обоих столбцах пропуски. Пустые значения свидетельствуют, что для некотрых заемщиков не доступна информация по общему трудовому стажу и ежемесячному доходу. Так же в столбце *days_employed* встречаются отрицательные значения общего трудового стажа, что невозможно. Причины пропусков и отрицательных значений могут быть разные: неверное заполнение таблиц, сбой в базе данных. Каждый отдельный случай необходимо разобрать и выявить причину. 

Значения в столбце *days_employed* не коррелириют с задачами исследования, так что заменим пропущенные значения в столбце на '0' и оставим отрицательные значения.

In [7]:
df['days_employed'] = df['days_employed'].fillna(0) 

Заполним пропуски в столбце *total_income* характерными значениями ежемесячного дохода, посчитанными в разных группах по типу занятости.

In [8]:
median_incomes = df.groupby('income_type')['total_income'].median().to_dict()
def filler_func(row):
    if pd.isna(row['total_income']):
        return median_incomes[row['income_type']]
    return row['total_income']
 
df['total_income'] = df.apply(filler_func, axis=1)

Убедимся, что таблица больше не содержит пропусков.

In [9]:
print(df.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Замена типа данных

Заменим вещественный тип данных на целочисленный методом .astype() c аргументом ('int'). Убедимся, что все заменили. 

In [10]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

Вызовем метод .value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке. Применим метод к столбцу *children*. 

In [11]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Дубликатов нет, но есть артефакты: количество детей -1 и 20. Удалим эти значения, так как их немного в общем объеме наблюдений. Проверим, удалились ли строки.

In [12]:
df = df[(df['children'] != -1) & (df['children'] != 20)].reset_index(drop=True)
print(df['children'].value_counts())

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


Применим метод .value_counts() к столбцу *dob_years*. 

In [13]:
df['total_income'].sort_values(ascending=True).tail(15)

13017    1128836
15127    1172459
19700    1223042
7411     1240165
2213     1278622
11013    1286280
15184    1350245
18250    1427934
18265    1551152
17403    1597613
17080    1711309
20692    1715018
9121     1726276
19494    2200852
12342    2265604
Name: total_income, dtype: int64

Дубликатов нет, но есть артефакты: возраст клиента - 0 лет. Удалим эти значения, так как их немного в общем объеме наблюдений. Проверим, удалились ли строки.

In [14]:
df = df[df['dob_years'] != 0].reset_index(drop=True)
print(df['dob_years'].value_counts())

35    614
41    603
40    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Применим метод .value_counts() к столбцу *education*. 

In [15]:
print(df['education'].value_counts())

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Значения уровня образования отличаются регистром. Чтобы учесть такие дубликаты, все символы в строке приведем к нижнему регистру вызовом метода .lower().

In [16]:
df['education'] = df['education'].str.lower()
print(df['education'].value_counts())

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64


Применим метод .value_counts() к столбцу *family_status*.

In [17]:
print(df['family_status'].value_counts())

женат / замужем          12254
гражданский брак          4139
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64


Дубликатов нет. Значения семейного положения приведем к нижнему регистру вызовом метода .lower().

In [18]:
df['family_status'] = df['family_status'].str.lower()
print(df['family_status'].value_counts())

женат / замужем          12254
гражданский брак          4139
не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64


Применим метод .value_counts() к столбцу *gender*.

In [19]:
print(df['gender'].value_counts())

F      14083
M       7218
XNA        1
Name: gender, dtype: int64


Дубликатов нет, но есть артефакты: пол клиента - XNA. Удалим это единственное значение. Проверим, удалилась ли строка.

In [20]:
df = df[df['gender'] != 'XNA']
print(df['gender'].value_counts())

F    14083
M     7218
Name: gender, dtype: int64


Проверим наличие дубликатов в таблице методом .duplicated(). В сочетании с методом sum() он возвращает количество дубликатов. Если найдутся, удалим, и проверим, все ли удалились.

In [21]:
print(df.duplicated().sum()) 

71


In [22]:
df = df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum()) 

0


Дубликатов нет. Данные пригодны для исследования.

### Выводы

На этапе предобработки в данных обнаружились пропуски, дубликаты, артефакты. Их удаление позволит провести анализ точнее. Поскольку сведения о ежемесячном доходе важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполняем пропущенные данные медиальным значением по типу занятости. Общий трудовой стаж не важен для исследования, оставляем этот столбец с мусором и заполняем пропущенные значения для удобства.

## Категоризация кредитных целей и клиентов <a id="step3"></a>  

### Лемматизация

Проведем лемматизацию (приведение слова к его словарной форме (лемме)) значений столбца *purpose*. Итоговый результат склеим вызовом метода join() в строку.

In [23]:
def lemmatisizer(row):
    return ' '.join(m.lemmatize(row['purpose']))
 
df['lemmatisized'] = df.apply(lemmatisizer, axis = 1)

### Категоризация

Работать с единичными отрывками и делать из них статистические выводы нельзя. Значит, нужно сгруппировать данные, чтобы численности каждой группы хватало для формулировки выводов. Нужна категоризация — объединение данных в категории. 

Запишем правила классификации клиентов как функции.

Вызовем метод .value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке. Применим метод к столбцу *lemmatisized*. 

In [24]:
print(df['lemmatisized'].value_counts())

автомобиль \n                                    961
свадьба \n                                       785
на   проведение   свадьба \n                     759
сыграть   свадьба \n                             755
операция   с   недвижимость \n                   669
покупка   коммерческий   недвижимость \n         655
покупка   жилье   для   сдача \n                 647
операция   с   коммерческий   недвижимость \n    643
операция   с   жилье \n                          641
покупка   жилье   для   семья \n                 636
жилье \n                                         635
покупка   жилье \n                               634
недвижимость \n                                  627
строительство   собственный   недвижимость \n    626
операция   со   свой   недвижимость \n           623
строительство   недвижимость \n                  619
покупка   свой   жилье \n                        618
строительство   жилой   недвижимость \n          617
покупка   недвижимость \n                     

Распределим клиентов по целям получения кредита:
* Клиенты, берущие кредит на покупку автомобиля, попадают в категорию «автомобиль»;
* Клиенты, берущие кредит на проведение свадьбы, - категория «свадьба»;
* Клиенты, берущие кредит на покупку/ремонт недвижимости (жилья), попадают в категорию «недвижимость»;
* Клиенты, берущие кредит на образование, - категория «образование».

Опишем функцию:

In [25]:
def purpose_group(row):
    purpose = ' '.join(m.lemmatize(row['purpose'])) 
    if 'автом' in purpose:
        return 'автомобиль'
    if 'свадь' in purpose:
        return  'свадьба'
    if 'образов' in purpose:
        return 'образование'
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'недвижимость'

Создадим отдельный столбец с категориями по целям получения кредита, и в его ячейках запишем значения, возвращаемые функцией.

In [26]:
df['purpose_category']= df.apply(purpose_group, axis=1)        

Распределим клиентов по количеству детей:
* Клиенты без детей (в столбце *children* значение «0») попадают в категорию «без детей»;
* Клиенты с детьми (в столбце *children* значения 1, 2, 3, 4, 5) — категория «есть дети».

На вход функции попадает количество детей, а возвращает она категорию клиента. Опишем функцию:

In [27]:
def сhildren_group(children):
    if children == 0:
        return 'без детей'
    if children > 0:
        return 'есть дети' 

Создадим отдельный столбец с категориями по детям, и в его ячейках запишем значения, возвращаемые функцией.

In [28]:
df['сhildren_category'] = df['children'].apply(сhildren_group)

Распределим клиентов по семейному положению: 
* Клиенты, живущие с партнером (в столбце *family_status* значения: женат / замужем; гражданский брак) - категория «в браке»;
* Клиенты, живущие без партнера (в столбце *family_status* значения: не женат / не замужем; в разводе; вдовец / вдова) - категория «не в браке».

На вход функции попадает семейное положение, а возвращает она категорию клиента. Опишем функцию:

In [29]:
married = ['женат / замужем', 'гражданский брак']
not_married = ['не женат / не замужем', 'в разводе', 'вдовец / вдова']

def family_status_group(family_status):
    if family_status in married:
        return 'в браке'
    if family_status in not_married:
        return 'не в браке' 

Создадим отдельный столбец с категориями по семейному положению, и в его ячейках запишем значения, возвращаемые функцией.

In [30]:
df['family_status_category'] = df['family_status'].apply(family_status_group)

Разобьем доход на промежутки с одинаковым количеством значений внутри каждого промежутка.

In [31]:
pd.qcut(df['total_income'],5).value_counts()

(20666.999, 98526.8]     4246
(98526.8, 132137.2]      4246
(132137.2, 161370.6]     4246
(161370.6, 214549.8]     4246
(214549.8, 2265604.0]    4246
Name: total_income, dtype: int64

Распределим клиентов по ежемесячному доходу: 
* Клиенты с доходом ниже 98 526.8 - категория «низкий»;
* Клиенты с доходом от 98 526.9 до 132 137.2 - категория «ниже среднего»;
* Клиенты с доходом от 132 137.3 до 161 370.6 - категория «средний»;
* Клиенты с доходом от 161 370.7 до 214 549.8 - категория «выше среднего»;
* Клиенты с доходом от 214 549.9 до 2 265 604.0 - категория «высокий».

На вход функции попадает ежемесячный доход, а возвращает она категорию клиента. Опишем функцию:

In [32]:
def income_group(income):
    if income <= 98526.8:
        return 'низкий'
    if income > 98526.8 and income <= 132137.2:
        return 'ниже среднего' 
    if income > 132137.2 and income <= 161370.6:
        return 'средний'
    if income > 161370.6 and income <= 214549.8:
        return 'выше среднего'
    return 'высокий'

Создадим отдельный столбец с категориями по ежемесячному доходу, и в его ячейках запишем значения, возвращаемые функцией.

In [33]:
df['total_income_category'] = df['total_income'].apply(income_group)

Распределим клиентов по задолжности: 
* Клиенты со значением в столбце *debt* «0» попадают в категорию «не должник»;
* Клиенты со значением в столбце *debt* «1» попадают в категорию «должник».

На вход функции попадает идентификатор задолжности по кредиту, а возвращает она категорию клиента. Опишем функцию:

In [34]:
def debt_group(debt):
    if debt == 0:
        return 'не должник'
    return 'должник' 

Создадим отдельный столбец с категориями по задолжности, и в его ячейках запишем значения, возвращаемые функцией.

In [35]:
df['debt_category'] = df['debt'].apply(debt_group)

Выведем первые 5 строк таблицы.

In [36]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatisized,purpose_category,сhildren_category,family_status_category,total_income_category,debt_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,недвижимость,есть дети,в браке,высокий,не должник
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль,есть дети,в браке,ниже среднего,не должник
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,недвижимость,без детей,в браке,средний,не должник
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование,есть дети,в браке,высокий,не должник
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба,без детей,в браке,средний,не должник


## Критерии оценки платежеспособности клиентов банка <a name="4"></a>  

### Гипотеза: разные цели кредита влияют на его возврат в срок

Была выдвинута гипотеза, что разные цели кредита влияют на его возврат в срок. Проверяем это предположение по данным о четырех категориях целей кредита — автомобиль, недвижимость, образование, свадьба.

In [37]:
df_purpose_pivot = df.pivot_table(index=['purpose_category'], columns='debt_category', values ='debt', aggfunc='count')
df_purpose_pivot.head()

debt_category,должник,не должник
purpose_category,,
автомобиль,397,3861
недвижимость,777,9926
образование,369,3601
свадьба,181,2118


Посчитаем процент невозврата.

In [38]:
df_purpose_pivot['ratio'] = (df_purpose_pivot['должник'] / (df_purpose_pivot['должник'] + df_purpose_pivot['не должник']) * 100)
df_purpose_pivot.sort_values(by='ratio', ascending=False).head()

debt_category,должник,не должник,ratio
purpose_category,,,
автомобиль,397,3861,9.323626
образование,369,3601,9.294710
свадьба,181,2118,7.872988
недвижимость,777,9926,7.259647


Выявлена зависимость наличия задолженности от цели кредитования:  
— наиболее высокий риск для целей "автомобиль" и "образование" ~9,3%  
— средний показатель у цели "свадьба" ~7,87%  
— самый низкий показатель у цели "недвижимость" ~7,25% 

### Гипотеза: есть зависимость между наличием детей и возвратом кредита в срок

Была выдвинута гипотеза, что надичие детей влияет на возврат кредиат в срок. Проверяем это предположение по данным о двух категориях клиентов — с детьми и без детей.

In [39]:
df_children_pivot = df.pivot_table(index=['сhildren_category'], columns='debt_category', values ='debt', aggfunc='count')
df_children_pivot.head()

debt_category,должник,не должник
сhildren_category,,
без детей,1058,12963
есть дети,666,6543


Посчитаем процент невозврата.

In [40]:
df_children_pivot['ratio'] = (df_children_pivot['должник'] / (df_children_pivot['должник'] + df_children_pivot['не должник']) * 100)
df_children_pivot.sort_values(by='ratio', ascending=False).head()

debt_category,должник,не должник,ratio
сhildren_category,,,
есть дети,666,6543,9.238452
без детей,1058,12963,7.545824


Из всех клиентов с детьми задолженность имели: 9,23%  
Из всех клиентов без детей задолженность имели: 7,54%  
  
Наличие детей может влиять на увеличение риска образования задоленности по кредиту.

### Гипотеза: есть зависимость между семейным положением и возвратом кредита в срок

Была выдвинута гипотеза, что семейное положение влияет на возврат кредита в срок. Проверяем это предположение по данным о двух категориях клиентов — в браке, не в браке.

In [41]:
df_family_status_pivot = df.pivot_table(index=['family_status_category'], columns='debt_category', values ='debt', aggfunc='count')
df_family_status_pivot.head()

debt_category,должник,не должник
family_status_category,,
в браке,1306,15019
не в браке,418,4487


Посчитаем процент невозврата.

In [42]:
df_family_status_pivot['ratio'] = (df_family_status_pivot['должник'] / (df_family_status_pivot['должник'] + df_family_status_pivot['не должник']) * 100)
df_family_status_pivot.sort_values(by='ratio', ascending=False).head()

debt_category,должник,не должник,ratio
family_status_category,,,
не в браке,418,4487,8.521916
в браке,1306,15019,8.000000


Процент задолженности среди клиентов, не состоящих в браке — 8,52% , выше, чем среди клиентов, состоящих в браке — 8,0%. Наличие официально оформленных семейных отношений может считаться положительным фактором для одобрения кредита.

### Гипотеза: есть зависимость между уровнем дохода и возвратом кредита в срок

Была выдвинута гипотеза, что уровень дохода влияет на возврат кредита в срок. Проверяем это предположение по данным о пяти категориях клиентов — доход низкий, ниже среднего, средний, выше среднего, высокий.

In [43]:
df_total_income_category_pivot = df.pivot_table(index=['total_income_category'], columns='debt_category', values ='debt', aggfunc='count')
df_total_income_category_pivot.head()

debt_category,должник,не должник
total_income_category,,
высокий,298,3948
выше среднего,356,3890
ниже среднего,356,3890
низкий,343,3903
средний,371,3875


Посчитаем процент невозврата.

In [44]:
df_total_income_category_pivot['ratio'] = (df_total_income_category_pivot['должник'] / (df_total_income_category_pivot['должник'] + df_total_income_category_pivot['не должник']) * 100)
df_total_income_category_pivot.sort_values(by='ratio', ascending=False).head()

debt_category,должник,не должник,ratio
total_income_category,,,
средний,371,3875,8.737635
выше среднего,356,3890,8.384362
ниже среднего,356,3890,8.384362
низкий,343,3903,8.078191
высокий,298,3948,7.018370


Уровень дохода не имеет значительного влияния: вне зависимости от уровня дохода процент задолженности в категориях "низкий", "ниже среднего", "средний" и "выше среднего" колеблится от 8,07% до 8,73%, только в случае "высокого" дохода процент в районе 7,01%.

## Результаты исследования <a name="5"></a>  

**Рабочие гипотезы**: 

* между наличием детей и возвратом кредита в срок есть зависимость;

* между семейным положением и возвратом кредита в срок есть зависимость;

* между уровнем дохода и возвратом кредита в срок есть зависимость;

* цели кредита влияют на его возврат в срок.

**Общие результаты**

Наличие детей в семье увеличивает вероятность образования задолженности.
  
Наличие статуса "женат/замужем" может положительно влиять на кредитную привлекательность клиента для банка, однако, не может иметь наибольший вес при оценке платежеспособности. Это фактор стоит учитывать в совокупности с наличием в семье детей, чтобы получить более точные оценки на этапе кредитного скоринга.
  
Задолженность не зависит от уровня дохода. В данном случае более показательными будут другие факторы: наличие детей, семейное положение и другие.  
   
Есть зависимость между целью кредита и риском образования задолженности — их необходимо учитывать при построении модели кредитного скоринга.